## Imports

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
cuda = torch.device("cuda")

## Loading Source Data

In [15]:
source_data = load_breast_cancer()

X_data = source_data.data
Y_data = source_data.target

## Data Preprocessing

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size = 0.25)

In [17]:
scale = StandardScaler()

In [18]:
# Scaling
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [19]:
#To tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(cuda)
Y_train = torch.tensor(Y_train, dtype=torch.float32).to(cuda)
X_test = torch.tensor(X_test, dtype=torch.float32).to(cuda)
Y_test = torch.tensor(Y_test, dtype=torch.float32).to(cuda)

## Neural Network

In [20]:
#define the neural network architecture

class NeuralNet(nn.Module):

  def __init__(self, input_nodes, hidden_nodes, output_nodes):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_nodes, hidden_nodes)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_nodes, output_nodes)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [21]:
#Hyperperameters
input_nodes = X_train.shape[1]
hidden_nodes = 64
output_nodes = 1
learning_rate = 0.001
epochs = 1000

In [22]:
model = NeuralNet(input_nodes, hidden_nodes, output_nodes).to(cuda)

## Training Parameters and Training Loop

In [23]:
# define loss and the optiizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
#Training loop
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, Y_train.view(-1, 1))
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        predicted = outputs.round()
        correct = (predicted == Y_train.view(-1,1)).float().sum()
        accuracy = correct/Y_train.size(0)
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {round(loss.item(), 5)}, Accuracy: {round(accuracy.item() * 100, 2)}%')

Epoch [100/1000], Loss: 0.11125, Accuracy: 97.89%
Epoch [200/1000], Loss: 0.06369, Accuracy: 98.59%
Epoch [300/1000], Loss: 0.04666, Accuracy: 99.06%
Epoch [400/1000], Loss: 0.03612, Accuracy: 99.3%
Epoch [500/1000], Loss: 0.028, Accuracy: 99.3%
Epoch [600/1000], Loss: 0.02152, Accuracy: 99.3%
Epoch [700/1000], Loss: 0.01669, Accuracy: 99.3%
Epoch [800/1000], Loss: 0.01309, Accuracy: 99.77%
Epoch [900/1000], Loss: 0.01033, Accuracy: 100.0%
Epoch [1000/1000], Loss: 0.00817, Accuracy: 100.0%


## Model evaluations

In [25]:
# evaluation on training set
model.eval()
with torch.no_grad():
    outputs = model(X_train)
    predicted = outputs.round()
    correct = (predicted == Y_train.view(-1,1)).float().sum()
    accuracy = correct/Y_train.size(0)
    print(f"Accuracy on training data: {round(accuracy.item() * 100, 2)}%")

Accuracy on training data: 100.0%


In [26]:
# evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    predicted = outputs.round()
    correct = (predicted == Y_test.view(-1,1)).float().sum()
    accuracy = correct/Y_test.size(0)
    print(f"Accuracy on test data: {round(accuracy.item() * 100, 2)}%")

Accuracy on test data: 97.9%
